In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL

In [3]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
elif host_name == 'jm-System-Product-Name':
    dir_train = ''
    dir_result = ''
else:
    raise NameError("You are running the script in a new computer, please define dir_train")

data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
featurepath_train= 'feature_e39_7999.npy'
data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
featurepath_test = 'feature_e39_7999.npy'
weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [4]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=featurepath_train, dir= dir_train, methods=[method])
x_test = np.load(dir_train+'\\'+data_to_fcpath_test)
w_test = np.load(dir_train+'\\'+weightpath)
bias = np.load(dir_train+'\\'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果


running method:  Mithral


In [6]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
np.save(dir_result+'\\fc1_256.npy', y_out_matmul)

io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[0.0034292  0.0022956  0.00382373 ... 0.00360621 0.00322708 0.00396036]
 [0.00573344 0.0033863  0.00363866 ... 0.0044085  0.00307089 0.00427151]
 [0.00432929 0.00396371 0.00331537 ... 0.00275869 0.00433741 0.00469637]
 ...
 [0.00497866 0.00402166 0.00245953 ... 0.0033775  0.00302241 0.00288731]
 [0.0046925  0.0037905  0.00317049 ... 0.00435381 0.00389608 0.00396245]
 [0.00299776 0.00482227 0.00244404 ... 0.00278139 0.0036241  0.0044476 ]]
(32, 256)
